In [13]:
from pathlib import Path

import torch

from examples.vits.text import text_to_sequence
from examples.vits.text.symbols import symbols
from examples.vits.torchy import utils, commons
from examples.vits.torchy.models import SynthesizerTrn
import IPython.display as ipd

def intersperse(lst, item):
  result = [item] * (len(lst) * 2 + 1)
  result[1::2] = lst
  return result


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

# TODO: auto download from drive https://drive.google.com/drive/folders/1ksarh-cJf3F5eKJjLVWY0X1j1qsQqiS2
FILENAME = "../weights/pretrained_ljs.pth"
if __name__ == '__main__':
  hps = utils.get_hparams_from_file("../configs/ljs_base.json")
  net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
  )
  _ = utils.load_checkpoint(FILENAME, net_g, None)
  stn_tst = get_text("Hello I am an AI created by Stan.", hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][
      0, 0].data.cpu().float().numpy()

  audio = ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False)
  ipd.display(audio)
  with open('test.wav', 'wb') as f:
    f.write(audio.data)


INFO:root:Loaded checkpoint '../weights/pretrained_ljs.pth' (iteration 0)
x: torch.Size([1, 91]), x_lengths: torch.Size([1]), sid: None

x: [[  0  50   0  83   0  54   0 156   0  57   0 135   0  16   0 156   0  43
    0 102   0  16   0  72   0  55   0  16   0  70   0  56   0  16   0 157
    0  47   0 102   0 156   0  43   0 102   0  16   0  53   0 123   0  51
    0 158   0 156   0  47   0 102   0 125   0 177   0  46   0  16   0  44
    0  43   0 102   0  16   0  61   0  62   0 156   0  72   0  56   0   4
    0]]
TextEncoder.forward1(x=[[  0  50   0  83   0  54   0 156   0  57   0 135   0  16   0 156   0  43
    0 102   0  16   0  72   0  55   0  16   0  70   0  56   0  16   0 157
    0  47   0 102   0 156   0  43   0 102   0  16   0  53   0 123   0  51
    0 158   0 156   0  47   0 102   0 125   0 177   0  46   0  16   0  44
    0  43   0 102   0  16   0  61   0  62   0 156   0  72   0  56   0   4
    0]])
TextEncoder.forward2(x=[[[ 0.00748267  0.01334275 -0.0089358  ...  0.00509426  0